In [ ]:
! тред умалчивает об исключения
если там обнаруживается ошибка, то тред умиирает и перестает работать, а программа продолжает свою работу

In [ ]:
# тут отлов исключения был непосредственно в самой ф-и, потому что в треде не получится его перехватить, даже если обернуть 
# сам тред в трай эксепт, т.к. после вызова треда управлене передается в основной поток
# отлов исключения по месту его возбуждения
import threading
import time

throw = False


def count():
    i = 0
    try:
        while True:
            if throw:
                raise ArithmeticError()

            i += 1
            print(f'{i=}')
            time.sleep(1)
    except ArithmeticError:
        print('en error occurred')


if __name__ == '__main__':
    print('main started')

    t = threading.Thread(target=count)
    t.start()

    time.sleep(5)
    throw = True
    for y in range(1, 10):
        print(f'{y=}')
        time.sleep(1)



но иногда надо прокидывать исключения на верхний уровень, чтобы именно тот решал, что делать с исключением
___
в вызывающем треде, который и исполняет код, где может возникнуть исключение, обработать исключение нельзя, можно это сделать, когда мы работаем с тредпулэкзекьютором
___
внизу первый пример будет без обращения к future - это тот результат, куда складывается результат исполнения ф-и с исключением

In [2]:
import concurrent.futures
import threading
import time


def div(divisor, limit):
    print(f'started div {divisor}')

    result = 0
    for x in range(1, limit):
        if x % divisor == 0:
            print(f'divisor= {divisor}, x= {x}')
            result += x
        time.sleep(0.2)

    print('exception is raised')
    raise Exception('smth went wrong')



if __name__ == '__main__':
    print('main started')

    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as ex:
        future = ex.submit(div, 3, 25)  # исключение запирается в future, пока обращения к future нет, искл не отработает
        time.sleep(3)

    print('main ended')


main started
started div 3
divisor= 3, x= 3
divisor= 3, x= 6
divisor= 3, x= 9
divisor= 3, x= 12
divisor= 3, x= 15
divisor= 3, x= 18
divisor= 3, x= 21
divisor= 3, x= 24
exception is raised
main ended


In [3]:
import concurrent.futures
import threading
import time


def div(divisor, limit):
    print(f'started div {divisor}')

    result = 0
    for x in range(1, limit):
        if x % divisor == 0:
            print(f'divisor= {divisor}, x= {x}')
            result += x
        time.sleep(0.2)

    print('exception is raised')
    #raise Exception('smth went wrong')  исключение закомменчено



if __name__ == '__main__':
    print('main started')

    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as ex:
        future = ex.submit(div, 3, 25)
        time.sleep(3)
        res = future.result()
    print('main ended')
# приложение завершило вою работу, а во втором - нет

main started
started div 3
divisor= 3, x= 3
divisor= 3, x= 6
divisor= 3, x= 9
divisor= 3, x= 12
divisor= 3, x= 15
divisor= 3, x= 18
divisor= 3, x= 21
divisor= 3, x= 24
exception is raised
main ended


In [4]:
import concurrent.futures
import threading
import time


def div(divisor, limit):
    print(f'started div {divisor}')

    result = 0
    for x in range(1, limit):
        if x % divisor == 0:
            print(f'divisor= {divisor}, x= {x}')
            result += x
        time.sleep(0.2)

    print('exception is raised')
    raise Exception('smth went wrong')  # исключение не закомменчено



if __name__ == '__main__':
    print('main started')

    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as ex:
        future = ex.submit(div, 3, 25)
        time.sleep(3)
        res = future.result()
    print('main ended')


main started
started div 3
divisor= 3, x= 3
divisor= 3, x= 6
divisor= 3, x= 9
divisor= 3, x= 12
divisor= 3, x= 15
divisor= 3, x= 18
divisor= 3, x= 21
divisor= 3, x= 24
exception is raised


Exception: smth went wrong

можно как_нибудь обработать это исключение, тогда основной поток не умрет, программа завершит работу

In [5]:
import concurrent.futures
import threading
import time


def div(divisor, limit):
    print(f'started div {divisor}')

    result = 0
    for x in range(1, limit):
        if x % divisor == 0:
            print(f'divisor= {divisor}, x= {x}')
            result += x
        time.sleep(0.2)

    print('exception is raised')
    raise Exception('smth went wrong')



if __name__ == '__main__':
    print('main started')

    with concurrent.futures.ThreadPoolExecutor(max_workers=1) as ex:
        future = ex.submit(div, 3, 25)
        time.sleep(3)
        try:
            res = future.result()
        except Exception as ex:
            print(repr(ex))
    print('main ended')


main started
started div 3
divisor= 3, x= 3
divisor= 3, x= 6
divisor= 3, x= 9
divisor= 3, x= 12
divisor= 3, x= 15
divisor= 3, x= 18
divisor= 3, x= 21
divisor= 3, x= 24
exception is raised
Exception('smth went wrong')
main ended


кроме простого Exception есть еще и другие исключения

1. первое - CancelledError - его суть в том, что если вызванная ф-я отменяется, то всплывает это исключение,

2. и есть еще TimeoutError, всплывает, если отведенное время для ф-и закончилось

In [6]:
import concurrent.futures
import threading
import time


def div(divisor, limit):
    print(f'started div {divisor}')

    result = 0
    for x in range(1, limit):
        if x % divisor == 0:
            print(f'divisor= {divisor}, x= {x}')
            result += x
        time.sleep(0.2)

    print('exception is raised')
    raise Exception('smth went wrong')


if __name__ == '__main__':
    print('started main')

    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as ex:
        res_list = ex.map(div, (3, 15), (4, 24))
        while res_list:
            try:
                cur_list = next(res_list)
            except StopIteration:
                print('stop iteration #1')
                break
            except Exception as ex:
                print(repr(ex), 'unhandled error')

    print('main ended')

started main
started div 3
started div 15
divisor= 3, x= 3
exception is raised
Exception('smth went wrong') unhandled error
stop iteration #1
divisor= 15, x= 15
exception is raised
main ended
